In [1]:
import xlwings as xw
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score


In [2]:
book = xw.Book("D:\\OneDrive\\nasdaq100.xlsx")
book.name

'nasdaq100.xlsx'

In [3]:
book.sheets

Sheets([<Sheet [nasdaq100.xlsx]预测值分析>, <Sheet [nasdaq100.xlsx]指数合理性判断>])

In [4]:
sheet1 = book.sheets['预测值分析']
sheet2 = book.sheets['指数合理性判断']

In [5]:
df1 = sheet1["A1"].expand().options(pd.DataFrame, index=True).value
df1

,收盘价
日期,
2014-12-31,4236.0
2015-01-31,4148.0
2015-02-28,4440.0
2015-03-31,4333.0
2015-04-30,4414.0
...,...
2024-07-31,19362.0
2024-08-31,19575.0
2024-09-30,20061.0


In [6]:
df1.index

DatetimeIndex(['2014-12-31', '2015-01-31', '2015-02-28', '2015-03-31',
               '2015-04-30', '2015-05-31', '2015-06-30', '2015-07-31',
               '2015-08-31', '2015-09-30',
               ...
               '2024-02-29', '2024-03-31', '2024-04-30', '2024-05-31',
               '2024-06-30', '2024-07-31', '2024-08-31', '2024-09-30',
               '2024-10-31', '2024-11-29'],
              dtype='datetime64[ns]', name='日期', length=120, freq=None)

In [7]:
df1.columns

Index(['收盘价'], dtype='object')

In [8]:
df1.dtypes

收盘价    float64
dtype: object

In [9]:
from pywander.math.excel import to_excel_int_time_v

print(to_excel_int_time_v(df1.index.to_pydatetime()))

[42004 42035 42063 42094 42124 42155 42185 42216 42247 42277 42308 42338
 42369 42400 42429 42460 42490 42521 42551 42582 42613 42643 42674 42704
 42735 42766 42794 42825 42855 42886 42916 42947 42978 43008 43039 43069
 43100 43131 43159 43190 43220 43251 43281 43312 43343 43373 43404 43434
 43465 43496 43524 43555 43585 43616 43646 43677 43708 43738 43769 43799
 43830 43861 43890 43921 43951 43982 44012 44043 44074 44104 44135 44165
 44196 44227 44255 44286 44316 44347 44377 44408 44439 44469 44500 44530
 44561 44592 44620 44651 44681 44712 44742 44773 44804 44834 44865 44895
 44926 44957 44985 45016 45046 45077 45107 45138 45169 45199 45230 45260
 45291 45322 45351 45382 45412 45443 45473 45504 45535 45565 45596 45625]


In [10]:

df1['int_time'] = to_excel_int_time_v(df1.index.to_pydatetime())
df1

,收盘价,int_time
日期,,
2014-12-31,4236.0,42004
2015-01-31,4148.0,42035
2015-02-28,4440.0,42063
2015-03-31,4333.0,42094
2015-04-30,4414.0,42124
...,...,...
2024-07-31,19362.0,45504
2024-08-31,19575.0,45535
2024-09-30,20061.0,45565


In [11]:
df1['收盘价'].to_numpy()

array([ 4236.,  4148.,  4440.,  4333.,  4414.,  4508.,  4396.,  4588.,
        4274.,  4181.,  4648.,  4664.,  4593.,  4279.,  4201.,  4483.,
        4341.,  4523.,  4417.,  4730.,  4771.,  4875.,  4801.,  4810.,
        4863.,  5116.,  5330.,  5436.,  5583.,  5788.,  5646.,  5880.,
        5988.,  5979.,  6248.,  6365.,  6396.,  6950.,  6854.,  6581.,
        6605.,  6967.,  7040.,  7232.,  7654.,  7627.,  6967.,  6949.,
        6330.,  6906.,  7097.,  7378.,  7781.,  7128.,  7671.,  7848.,
        7691.,  7749.,  8083.,  8418.,  8727.,  8991.,  8461.,  7813.,
        9000.,  9555., 10157., 10906., 12111., 11418., 11053., 12268.,
       12888., 12925., 12909., 13091., 13861., 13687., 14555., 14960.,
       15583., 14690., 15851., 16136., 16320., 14930., 14238., 14839.,
       12855., 12642., 11504., 12948., 12272., 10971., 11406., 12030.,
       10940., 12102., 12042., 13181., 13246., 14254., 15179., 15757.,
       15501., 14715., 14410., 15948., 16826., 17137., 18044., 18246.,
      

In [12]:
df1['int_time'].to_numpy()

array([42004, 42035, 42063, 42094, 42124, 42155, 42185, 42216, 42247,
       42277, 42308, 42338, 42369, 42400, 42429, 42460, 42490, 42521,
       42551, 42582, 42613, 42643, 42674, 42704, 42735, 42766, 42794,
       42825, 42855, 42886, 42916, 42947, 42978, 43008, 43039, 43069,
       43100, 43131, 43159, 43190, 43220, 43251, 43281, 43312, 43343,
       43373, 43404, 43434, 43465, 43496, 43524, 43555, 43585, 43616,
       43646, 43677, 43708, 43738, 43769, 43799, 43830, 43861, 43890,
       43921, 43951, 43982, 44012, 44043, 44074, 44104, 44135, 44165,
       44196, 44227, 44255, 44286, 44316, 44347, 44377, 44408, 44439,
       44469, 44500, 44530, 44561, 44592, 44620, 44651, 44681, 44712,
       44742, 44773, 44804, 44834, 44865, 44895, 44926, 44957, 44985,
       45016, 45046, 45077, 45107, 45138, 45169, 45199, 45230, 45260,
       45291, 45322, 45351, 45382, 45412, 45443, 45473, 45504, 45535,
       45565, 45596, 45625])

## 趋势线模拟为什么选择指数函数
认为其有一个内在的增加动力，随着时间有着一定的增长率，那么趋势线模拟可以选用指数函数。



![img](趋势线指数规律.png)

[4.46909722e-05 4.36328714e+00]
[[ 8.03146639e-10 -4.02322325e-06]
 [-4.02322325e-06  2.01608532e-02]]


0.9104783660005434

In [13]:
xdata = df1['int_time'].to_numpy()
ydata = df1['收盘价'].to_numpy()
xdata = xdata.astype(np.float64)

def func(x, a, b):
    # 因为np.exp接受过大的数会造成数值溢出 
    # 必须先改造输入xdata 让其变得小一些
    return a * np.exp(b * (x * 0.0001))

fitting_opts = {
    #'maxfev': 10,  # 设置最大迭代次数
    #'method': 'lm'
}

popt, pcov = curve_fit(func, xdata, ydata , **fitting_opts)
print(popt)
print(pcov)

# 实际曲线
plt.plot(xdata, ydata, 'b-', label='data')

# 预测曲线
predict_ydata = func(xdata, *popt)
plt.plot(xdata, predict_ydata, 'r-')

r2_score(ydata, predict_ydata)

In [14]:
book.names['Var_1'].delete()
book.names['Var_2'].delete()

In [15]:
book.names.add("Var_1", popt[0])
book.names.add("Var_2", popt[1])
print(book.names)

[<Name 'Var_1': =0.0000446909721765957>, <Name 'Var_2': =4.36328714246767>]


In [16]:
book.save()
#app = book.app
#%app.quit()